In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle

exp_1 = pd.read_excel("EXP-00001-Master.xlsx")
exp_2 = pd.read_excel('EXP-00002-Master.xlsx')
exp_3 = pd.read_excel('EXP-00003-Master.xlsx')
exp_4 = pd.read_excel('EXP-00004-Master.xlsx')
exp_5 = pd.read_excel('EXP-00004-Master.xlsx')


exp_1.drop(index=0, inplace=True)
exp_2.drop(index=0, inplace=True)
exp_3.drop(index=0, inplace=True)
exp_4.drop(index = 0, inplace = True)
exp_5.drop(index = 0, inplace= True)

exp_1.reset_index(drop=True, inplace=True)
exp_2.reset_index(drop=True, inplace=True)
exp_3.reset_index(drop=True, inplace=True)
exp_4.reset_index(drop=True, inplace=True)
exp_5.reset_index(drop=True, inplace=True)

In [ ]:
not_included = ['Id', 'Filter0','Filter1', 'Filter2','Filter3', 'Filter4', 'Filter5', 'Filter6', 'hash', 'Img Id', 'Curvature', 'Transparency', 'Angularity']
filtered = [x for x in exp_1.columns if x not in not_included]

exp_1_filtered = exp_1[filtered]
exp_2_filtered = exp_2[filtered]
exp_3_filtered = exp_3[filtered]
exp_4_filtered = exp_4[filtered]
exp_5_filtered = exp_5[filtered]

exp_1_filtered['Production Stage'] = 0
exp_2_filtered['Production Stage'] = 1
exp_3_filtered['Production Stage'] = 2
exp_4_filtered['Production Stage'] = 3
exp_5_filtered['Production Stage'] = 4

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data[filtered],
    data['Production Stage'],
    test_size=0.1,
    stratify= data['Production Stage'],
    random_state=44)

y_train.groupby(y_train).count()
y_test.groupby(y_test).count()

In [ ]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

GBModel = GradientBoostingClassifier(
                                    n_estimators=100, 
                                    learning_rate = .30 , 
                                    max_depth = 3, 
                                    max_features= 'auto',
                                    criterion= 'mse',
                                    loss = 'deviance'
                                    )


GBModel.fit(scale(data[filtered].values), data["Production Stage"])

# cluster = GBModel.predict([scale(data[filtered].values)[0]])[0]

random = data.sample(frac = 1)

ten_thousand = random[:10000]

# predictions = model.predict(two_hundred[filtered])

predictionsGB = GBModel.predict(ten_thousand[filtered])

print('Accuracy score:',accuracy_score(ten_thousand['Production Stage'], predictionsGB))

In [ ]:
from sklearn.pipeline import Pipeline
gbc_pipe = Pipeline([('GBC', GradientBoostingClassifier())])

params = [{ 'max_depth': [ 1, 2, 3, 4, 5],
            'learning_rate': [.01, .10, .20, .30, .40, .50],
            'n_estimators': [50, 150, 250, 350, 450,],
            'loss': ['log_loss', 'deviance', 'exponential'],
            'criterion': ['friedman_mse', 'squared_error', 'mse']
            }]

search = GridSearchCV(GBModel, 
                    param_grid = params,
                    scoring = 'accuracy',
                    cv = 5)

search.fit(X_train,y_train)
print(search.best_params_)

print("score: {}".format(search.score(X_train, y_train)))
